### Imports

In [1]:
import os
import pandas as pd
import json
import folium
import matplotlib.pyplot as plt
import numpy as np
import re
import math
import numpy as np

### Constants

In [2]:
DATA_FOLDER = 'data_clean'
IS_DATE = re.compile("^[1-2]{1}[0-9]{3}$")

### Load Dataframe

In [3]:
def describeDf(df,name="DESCRIPTION",level=1):
    print("_________________ " + name+" _________________")
    date = []
    if(level==1):
        print("\n")
        for col in df.columns:
            if((not IS_DATE.match(col)) & (not re.compile("[0-9]+ an").match(col))):
                print("         ATTRIBUTE: "+col)
                print("   "+str(df[col].unique()))
                print("\n")
            else:
                date.append(col)
        print("         VALUES: "+str(date))
        print("\n\n\n\n")
    else :
        print(df.columns.values)
        print("\n\n\n\n")

print("Dataframes available:\n\n")
for file in os.listdir(DATA_FOLDER):
    filename = os.fsdecode(file)
    tablename = "df_"+os.path.splitext(os.path.splitext(filename)[0])[0]
    globals()[tablename] = pd.read_csv(DATA_FOLDER + '/' + filename)
    globals()[tablename] = globals()[tablename].drop(columns=['Unnamed: 0'])
    describeDf(globals()[tablename], name=tablename,level=0)
    

Dataframes available:


_________________ df_Accident_cantons _________________
['canton_index' 'canton' 'gravite_index' 'gravite' 'ruralite_index'
 'ruralite' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999' '2000'
 '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009' '2010'
 '2011' '2012' '2013' '2014' '2015' '2016' '2017']





_________________ df_Accident_circonstances _________________
['mois_index' 'mois' 'day_index' 'day' 'hours_index' 'hours'
 'gravite_index' 'gravite' 'ruralite_index' 'ruralite' 'contexte_index'
 'contexte' 'circonstance_index' 'circonstance' '1992' '1993' '1994'
 '1995' '1996' '1997' '1998' '1999' '2000' '2001' '2002' '2003' '2004'
 '2005' '2006' '2007' '2008' '2009' '2010' '2011' '2012' '2013' '2014'
 '2015' '2016' '2017']





_________________ df_Accident_objets _________________
['voiture_index' 'voiture' 'age_index' 'age' 'sexe_index' 'sexe'
 'annee_permis_index' 'annee_permis' 'faute_index' 'faute' 'gravite_index'
 'gravite' '1992' '19

### Describe a DataFrame

In [4]:
describeDf(df_Accident_victimes)

_________________ DESCRIPTION _________________


         ATTRIBUTE: voiture_index
   [210. 211. 214. 220. 221. 222. 223. 224. 227. 228. 229.]


         ATTRIBUTE: voiture
   ['Voiture de tourisme' 'Véhicule de transport de personnes'
 'Véhicule de transport de choses' 'Cycle' 'Cyclomoteur' 'Motocycle léger'
 "Motocycle jusqu'à 125 cm3" 'Motocycle de plus de 125 cm3' 'Piéton'
 'Autre véhicule non motorisé' 'Autre et inconnu']


         ATTRIBUTE: sexe_index
   [1. 2. 9.]


         ATTRIBUTE: sexe
   ['Homme' 'Femme' 'Inconnu']


         ATTRIBUTE: role_index
   [300. 301. 303.]


         ATTRIBUTE: role
   ['Conducteur' 'Piéton' 'Passager']


         ATTRIBUTE: age_index
   [  0.   5.   6.  10.  15.  18.  20.  21.  30.  40.  50.  60.  70. 999.]


         ATTRIBUTE: age
   ["jusqu'à 4 ans" '5 ans' '6-9 ans' '10-14 ans' '15-17 ans' '18-19 ans'
 '20 ans' '21-29 ans' '30-39 ans' '40-49 ans' '50-59 ans' '60-69 ans'
 '70 ans et +' 'Inconnu']


         ATTRIBUTE: consequence_index
  

## A. Prepare Data

### A.1. Get the Swiss population

In [5]:
def build_age(intervales):
    ages_total = []
    for i in range(len(intervales)-1):
        ages = []
        ages.append(intervales[i])
        for k in range(int(intervales[i]),int(intervales[i+1])):
            if(k != 1 and k<100):
                ages.append('{} ans'.format(k))
            if(k == 1):
                ages.append('{} an'.format(k))
            if(k==100):
                ages.append('100 ans ou plus')
                break
        ages_total.append(ages)
    return ages_total

In [6]:
intervales = build_age(df_Accident_objets.age_index.unique())
for elem in intervales:
    df_Population_canton[str(elem[0])] = df_Population_canton.loc[:,elem[1:]].sum(axis=1)
toKeep = []
for elem in intervales :
    toKeep.append(str(elem[0]))
toKeep.append('annee')
toKeep.append('sexe')

df_pop_Suisse = df_Population_canton[(df_Population_canton['canton'] == 'Suisse') & (df_Population_canton['sexe'] != 'Sexe - Total')\
                                     & (df_Population_canton.etat_civil == "Etat civil - Total")]

df_pop_Suisse = df_pop_Suisse[toKeep]
df_pop_Suisse['annee'] = df_pop_Suisse['annee'].map(int).map(str)
df_pop_Suisse = df_pop_Suisse.groupby(['annee','sexe']).sum()
df_pop_Suisse = df_pop_Suisse.transpose()
df_pop_Suisse.index = df_pop_Suisse.index.map(float)
display(df_pop_Suisse.head(2))

annee      2010                2011                2012                2013  \
sexe      Femme     Homme     Femme     Homme     Femme     Homme     Femme   
0.0    191197.0  202097.0  195313.0  206364.0  198173.0  210120.0  201280.0   
5.0     37402.0   39653.0   37807.0   40014.0   38743.0   40693.0   39450.0   

annee                2014                2015                2016            \
sexe      Homme     Femme     Homme     Femme     Homme     Femme     Homme   
0.0    213036.0  204828.0  216474.0  207157.0  219664.0  210312.0  222765.0   
5.0     42127.0   40359.0   42763.0   41844.0   43688.0   41177.0   43629.0   

annee      2017            
sexe      Femme     Homme  
0.0    212201.0  224251.0  
5.0     41704.0   44528.0

> `df_pop_Suisse` contains the Swiss population by **year** and **sexe** on *columns* and by **age** range on *rows*

### A.2. Normalize the data by the population

In [7]:
df_Accident_victimes_normalized = df_Accident_victimes.copy()
df_Accident_victimes_normalized = df_Accident_victimes_normalized[df_Accident_victimes_normalized['sexe_index']!=9]
for year,sexe in df_pop_Suisse.columns.values:
    for age in df_pop_Suisse.index.values:
        df_Accident_victimes_normalized.loc[(df_Accident_victimes_normalized.age_index==age) &\
                                           (df_Accident_victimes_normalized.sexe==sexe),year]\
        = df_Accident_victimes_normalized[year]*10000 / df_pop_Suisse.loc[df_pop_Suisse.index==age,year][sexe].values[0]
reg = re.compile("200[0-9]|1[0-9]{3}")
toDrop = []
for elem in df_Accident_victimes_normalized.columns:
    if reg.match(elem):
        toDrop.append(elem)
df_Accident_victimes_normalized = df_Accident_victimes_normalized.drop(columns=toDrop)
df_Accident_victimes_normalized = df_Accident_victimes_normalized[df_Accident_victimes_normalized.age_index != 999]
display(df_Accident_victimes_normalized.head(2))

,voiture_index,voiture,sexe_index,sexe,role_index,role,age_index,age,consequence_index,consequence,2010,2011,2012,2013,2014,2015,2016,2017
0,210.0,Voiture de tourisme,1.0,Homme,300.0,Conducteur,0.0,jusqu'à 4 ans,315.0,Blessé léger,0.0,0.0,0.047592,0.0,0.0,0.0,0.0,0.0
1,210.0,Voiture de tourisme,1.0,Homme,300.0,Conducteur,0.0,jusqu'à 4 ans,316.0,Blessé grave,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


> `df_Accident_acteurs_normalized` contains the proportion in term of **age range** and **sexe** of the population involved in accident

In [8]:
df_Accident_objets_normalized = df_Accident_objets.copy()
df_Accident_objets_normalized = df_Accident_objets_normalized[df_Accident_objets_normalized['sexe_index']!=9]
for year,sexe in df_pop_Suisse.columns.values:
    for age in df_pop_Suisse.index.values:
        df_Accident_objets_normalized.loc[(df_Accident_objets_normalized.age_index==age) &\
                                           (df_Accident_objets_normalized.sexe==sexe),year]\
        = df_Accident_objets_normalized[year]*10000 / df_pop_Suisse.loc[df_pop_Suisse.index==age,year][sexe].values[0]
reg = re.compile("200[0-9]|1[0-9]{3}")
toDrop = []
for elem in df_Accident_objets_normalized.columns:
    if reg.match(elem):
        toDrop.append(elem)
df_Accident_objets_normalized = df_Accident_objets_normalized.drop(columns=toDrop)
df_Accident_objets_normalized = df_Accident_objets_normalized[df_Accident_objets_normalized.age_index != 999]
display(df_Accident_objets_normalized.head(2))

,voiture_index,voiture,age_index,age,sexe_index,sexe,annee_permis_index,annee_permis,faute_index,faute,gravite_index,gravite,2010,2011,2012,2013,2014,2015,2016,2017
0,210.0,Voiture de tourisme,0.0,Jusqu'à 4 ans,1.0,Homme,0.0,0 an,0.0,Objet sans faute ni influence,315.0,Accidents avec blessés légers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,210.0,Voiture de tourisme,0.0,Jusqu'à 4 ans,1.0,Homme,0.0,0 an,0.0,Objet sans faute ni influence,316.0,Accidents avec blessés graves,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


> `df_Accident_objets_normalized` contains the proportion in term of **age range** and **sexe** of the population involved in accident

## B. Analyse on the actors of accidents

### B.1. Sexe and Age of conductors

##### Using `df_Accident_objets_normalized`

In [9]:
df_conducteur = df_Accident_objets_normalized[(df_Accident_objets_normalized.annee_permis_index!=0)\
                                           & (df_Accident_objets_normalized.annee_permis_index!=999)\
                                           & (df_Accident_objets_normalized.voiture_index==210)]
conducteur = df_conducteur.groupby(['sexe','age']).sum().sort_values(['2010'], ascending=False)[[col for col in df_conducteur.columns if IS_DATE.match(col)]]
display(conducteur.head(5))


2010       2011       2012       2013       2014  \
sexe  age                                                                
Homme 20 ans     60.862079  56.936537  53.686185  48.666776  48.905259   
      21-29 ans  57.985585  54.012681  48.313519  45.961388  42.669766   
      30-39 ans  43.948825  41.711458  40.272102  38.930899  36.923781   
      40-49 ans  42.762340  38.157735  37.095152  34.921224  33.867883   
      50-59 ans  40.635182  35.750283  34.096272  32.277577  32.620497   

                      2015       2016       2017  
sexe  age                                         
Homme 20 ans     36.807831  44.557734  41.966548  
      21-29 ans  41.561941  41.601469  39.339320  
      30-39 ans  33.857228  34.213227  34.741186  
      40-49 ans  32.116146  31.554196  31.349074  
      50-59 ans  32.241042  31.179018  29.243624

##### Using `df_Accident_victimes_normalized`

In [10]:
conducteur = df_Accident_victimes_normalized[((df_Accident_victimes_normalized['role_index']==300)\
                                            | (df_Accident_victimes_normalized['role_index']==303))\
                                            & (df_Accident_victimes_normalized.voiture_index==210)]\
            .groupby(['sexe','age']).sum().sort_values(['2010'], ascending=False)\
            [[col for col in conducteur.columns if IS_DATE.match(col)]]
display(conducteur.head(5))

2010       2011       2012       2013       2014  \
sexe  age                                                                
Homme 20 ans     44.445334  46.223739  41.711571  31.490267  32.535298   
      18-19 ans  44.200595  46.348738  42.238202  32.712712  32.365189   
Femme 20 ans     41.437615  30.051191  36.796626  33.675057  33.946373   
      21-29 ans  32.622508  30.017217  26.398688  26.688693  25.124594   
      18-19 ans  32.233110  29.373226  34.010571  26.050984  24.274007   

                      2015       2016       2017  
sexe  age                                         
Homme 20 ans     31.954051  31.912972  26.153066  
      18-19 ans  28.816360  26.981696  27.167857  
Femme 20 ans     28.860337  30.246874  31.206355  
      21-29 ans  24.767878  24.340530  23.769673  
      18-19 ans  23.718848  26.664558  22.312745

> RESULTS ARE NOT THE SAME :(

### C) Test of global population consistence among different datasets

In this section, unknown values were not excluded to avoid asymetric operations on different datasets. 

In [11]:
#test dataset with cantons
df_acc_cantons = df_Accident_cantons

#only taking years from 2010 to 2017
toDrop = []
for elem in df_acc_cantons.columns:
    if reg.match(elem):
        toDrop.append(elem)
df_acc_cantons = df_acc_cantons.drop(columns=toDrop)

#aggregate groups by gravite 
acc_cantons = df_acc_cantons.groupby(['gravite']).sum().sort_values(['2010'], ascending=False)\
            [[col for col in df_acc_cantons.columns if IS_DATE.match(col)]]
display(acc_cantons)

,2010,2011,2012,2013,2014,2015,2016,2017
gravite,,,,,,,,
Accidents avec blessés légers,15214.0,14568.0,13980.0,13357.0,13756.0,13886.0,13821.0,14153.0
Accidents avec blessés graves,4082.0,4110.0,3867.0,3859.0,3818.0,3612.0,3548.0,3427.0
Accidents mortels,313.0,312.0,301.0,257.0,229.0,238.0,208.0,219.0


In [12]:
#test dataset with circonstances
df_acc_circo = df_Accident_circonstances

#only taking years from 2010 to 2017
toDrop = []
for elem in df_acc_circo.columns:
    if reg.match(elem):
        toDrop.append(elem)
df_acc_circo = df_acc_circo.drop(columns=toDrop)

#aggregate groups by gravite
acc_circo = df_acc_circo.groupby(['gravite']).sum().sort_values(['2010'], ascending=False)\
            [[col for col in df_acc_circo.columns if IS_DATE.match(col)]]
display(acc_circo)

,2010,2011,2012,2013,2014,2015,2016,2017
gravite,,,,,,,,
Accidents avec blessés légers,10605.0,10383.0,9700.0,9262.0,9681.0,9681.0,9708.0,9696.0
Accidents avec blessés graves,2460.0,2485.0,2291.0,2326.0,2245.0,2128.0,2079.0,1934.0
Accidents mortels,174.0,185.0,170.0,146.0,139.0,129.0,128.0,124.0


In [13]:
#test dataset with route type
df_acc_routeType = df_Accident_type_route

#only taking years from 2010 to 2017
toDrop = []
for elem in df_acc_routeType.columns:
    if reg.match(elem):
        toDrop.append(elem)
df_acc_routeType = df_acc_routeType.drop(columns=toDrop)

#aggregate groups by gravite
acc_routeType = df_acc_routeType.groupby(['gravite']).sum().sort_values(['2010'],ascending=False)\
            [[col for col in df_acc_routeType.columns if IS_DATE.match(col)]]
display(acc_routeType)

,2010,2011,2012,2013,2014,2015,2016,2017
gravite,,,,,,,,
Accidents avec blessés légers,15214.0,14568.0,13980.0,13357.0,13756.0,13886.0,13821.0,14153.0
Accidents avec blessés graves,4082.0,4110.0,3867.0,3859.0,3818.0,3612.0,3548.0,3427.0
Accidents mortels,313.0,312.0,301.0,257.0,229.0,238.0,208.0,219.0


In [14]:
#test dataset with objects
df_acc_objet = df_Accident_objets

#only taking years from 2010 to 2017
toDrop = []
for elem in df_acc_objet.columns:
    if reg.match(elem):
        toDrop.append(elem)
df_acc_objet = df_acc_objet.drop(columns=toDrop)

#aggregate groups by gravite 
acc_objet = df_acc_objet.groupby(['gravite']).sum().sort_values(['2010'],ascending=False)\
            [[col for col in df_acc_objet.columns if IS_DATE.match(col)]]
display(acc_objet)

,2010,2011,2012,2013,2014,2015,2016,2017
gravite,,,,,,,,
Accidents avec blessés légers,29117.0,28122.0,26720.0,25586.0,26380.0,26428.0,26392.0,26802.0
Accidents avec blessés graves,7165.0,7146.0,6774.0,6854.0,6635.0,6242.0,6158.0,5884.0
Accidents mortels,564.0,563.0,555.0,449.0,411.0,414.0,375.0,392.0


We know that in 2017, there were 17'799 accidents causing body injuries in Switzerland. Therefore, it means that cantons and routeType datasets are correct! 
However, the dataset of circomstances is lacking of values (missing ~1/2) and the one about objects is containing too much (round 2 times more accidents). **This might be explained by the fact that multiple objects are involved in one accident.**

In [15]:
#aggregate objets dataset in groups by vehicle type
acc_objet = df_acc_objet.groupby(['voiture']).sum().sort_values(['2010'],ascending=False)\
            [[col for col in df_acc_objet.columns if IS_DATE.match(col)]]
display(acc_objet)

,2010,2011,2012,2013,2014,2015,2016,2017
voiture,,,,,,,,
Voiture de tourisme,22889.0,21624.0,20601.0,19894.0,19717.0,19238.0,19371.0,18981.0
Cycle,3373.0,3681.0,3520.0,3545.0,4015.0,4230.0,4296.0,4481.0
Piéton,2648.0,2577.0,2490.0,2478.0,2441.0,2435.0,2478.0,2255.0
Motocycle de plus de 125 cm3,2182.0,2441.0,2246.0,2035.0,2260.0,2302.0,2123.0,2225.0
Véhicule de transport de choses,2049.0,2021.0,1940.0,1945.0,1931.0,2045.0,1946.0,2058.0
Motocycle jusqu'à 125 cm3,1553.0,1077.0,1026.0,981.0,1019.0,928.0,835.0,946.0
Motocycle léger,622.0,932.0,793.0,696.0,726.0,625.0,567.0,652.0
Cyclomoteur,616.0,579.0,470.0,444.0,413.0,405.0,409.0,393.0
Autre et inconnu,454.0,374.0,442.0,437.0,397.0,365.0,384.0,387.0


In previous table, we can see that in 2017, more cars were involved in accidents than the total number of accidents (18981>17799). This illustrates that many vehicles are implied in accidents. 

In [16]:
totCrash=sum(acc_cantons['2017'])
totObjects = sum(acc_objet['2017'])
objetPerCrash = totObjects/totCrash

print('In average, a crash was involving %.2f objects in 2017.' % objetPerCrash)

In average, a crash was involving 1.86 objects in 2017.


Now let's study the case of circonstances dataset:

In [17]:
#aggregate groups by circonstances
acc_circo = df_acc_circo.groupby(['circonstance']).sum().sort_values(['2010'], ascending=False)\
            [[col for col in df_acc_circo.columns if IS_DATE.match(col)]]
display(acc_circo)

,2010,2011,2012,2013,2014,2015,2016,2017
circonstance,,,,,,,,
Autre facteur d'influence,12446.0,12182.0,11400.0,11038.0,11430.0,11293.0,11346.0,11195.0
Alcool,658.0,709.0,603.0,555.0,491.0,491.0,439.0,411.0
Drogues,63.0,71.0,59.0,46.0,51.0,61.0,53.0,71.0
Alcool + droques,28.0,30.0,43.0,29.0,38.0,31.0,35.0,31.0
Médicaments,23.0,38.0,29.0,34.0,34.0,41.0,31.0,29.0
Alcool + médicaments,17.0,15.0,23.0,25.0,13.0,14.0,9.0,11.0
Drogues et médicaments,3.0,5.0,3.0,6.0,7.0,5.0,1.0,5.0
Alcool + drogues + médicaments,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0


In [18]:
totCirco = sum(acc_circo['2017'])
totCirco

11754.0

Hmmm... it looks like this dataset is incomplete! It provides only 11754 values out of 17799 accidents... Another possibility would be that "Autres facteurs d'influence" doesn't contain "No influence factor" for example. 

In [19]:
#aggregate circo dataset in groups by context
acc_circo = df_acc_circo.groupby(['contexte']).sum().sort_values(['2010'], ascending=False)\
            [[col for col in df_acc_circo.columns if IS_DATE.match(col)]]
display(acc_circo)

,2010,2011,2012,2013,2014,2015,2016,2017
contexte,,,,,,,,
Tamponnement,4061.0,4062.0,3822.0,3659.0,3882.0,3701.0,3862.0,3651.0
Changement de direction (en obliquant),3887.0,3855.0,3457.0,3383.0,3535.0,3481.0,3438.0,3464.0
Accident concernant des piétons,2251.0,2187.0,2113.0,2100.0,2081.0,2110.0,2073.0,2097.0
À une bifurcation sans changement de direction,1058.0,1027.0,979.0,922.0,916.0,910.0,881.0,865.0
Lors de croisement (longitudinal),833.0,805.0,762.0,675.0,649.0,665.0,655.0,703.0
Autre,602.0,546.0,522.0,489.0,497.0,537.0,530.0,479.0
Dépassement,465.0,497.0,415.0,434.0,416.0,435.0,402.0,417.0
Avec un animal,82.0,74.0,91.0,72.0,89.0,99.0,74.0,78.0


Still the same issue, and 'Autre' is even included... I think there miss values